# Stokes example based on Julian's PETSc example.

In [1]:
import petsc4py.PETSc as PETSc
import underworld3 as uw
import numpy as np

## Create Mesh

In [2]:
mesh = uw.FeMesh(elementRes=[20,20], minCoords=(-1.0, -1.0), maxCoords=(1.0, 1.0), simplex=True)
mesh.save("mesh.h5")

In [3]:
snes = PETSc.SNES().create(comm=mesh.plex.comm)
snes.setDM(mesh.plex)

## Setup Discretisation DM

## Velocity

In [4]:
# Get Communicator from DM
comm = mesh.plex.comm
# Create Default FE
fe_vel = PETSc.FE().createDefault(dim=mesh.dim, nc=2, isSimplex=mesh.isSimplex, qorder=-1, comm=comm)
# Set Name
fe_vel.setName("velocity")
# Get Quadrature
fe_vel_quad = fe_vel.getQuadrature()
# Get Face Quadrature
fe_vel_face_quad = fe_vel.getFaceQuadrature()

## Pressure

In [5]:
# Create Default FE
fe_pres = PETSc.FE().createDefault(dim=mesh.dim, nc=1, isSimplex=mesh.isSimplex, qorder=-1, comm=comm)
# Set Name
fe_pres.setName("pressure")
# Set Quadrature
fe_pres.setQuadrature(fe_vel_quad)
# Set Face Quadrature
fe_pres.setFaceQuadrature(fe_vel_face_quad)

In [6]:
# Set discretisation and boundary conditions for each mesh
# Must set the fields before creating the section
# If you don't create a section, we can't get a vector
mesh.plex.setField(0, fe_vel)
mesh.plex.setField(1, fe_pres)
origSect = mesh.plex.createSection([2,1], [2,0,0,0,0,1])
mesh.plex.setDefaultSection(origSect)
mesh.plex.createDS()

## Set up saddle point system, aka stokes

In [7]:
from underworld3.stokes import StokesSetup

In [8]:
StokesSetup(mesh.plex)

## Auxiliary Fields

### density and viscosity

In [9]:
fe_density = PETSc.FE().createDefault(dim=mesh.dim, nc=1, isSimplex=mesh.isSimplex, qorder=-1, comm=comm)
fe_density.setName("density")
fe_density.setQuadrature(fe_vel_quad)
fe_density.setFaceQuadrature(fe_vel_face_quad)

fe_viscosity = PETSc.FE().createDefault(dim=mesh.dim, nc=1, isSimplex=mesh.isSimplex, qorder=-1, comm=comm)
fe_viscosity.setName("viscosity")
fe_viscosity.setQuadrature(fe_vel_quad)
fe_viscosity.setFaceQuadrature(fe_vel_face_quad)

#### Set up auxiliary DM

In [10]:
coords = mesh.plex.getCoordinates()
dmAux = mesh.plex.clone()
mesh.plex.compose("dmAux", dmAux)

In [11]:
dmAux.setCoordinates(coords)
dmAux.setField(0, fe_density)
dmAux.setField(1, fe_viscosity)
section = mesh.plex.createSection([1,1], [1,0,0,1,0,0])
dmAux.setSection(section)
dmAux.createDS()

#### Set up auxiliary fields

In [12]:
A = dmAux.createLocalVector()

# Get Coordinates
coords_vec = dmAux.getCoordinates()

nf = dmAux.getNumFields()

R = coords_vec.array[0::nf]**2 + coords_vec.array[1::nf]**2
densities = np.where(R < 0.2**2, 1.5, 0.1) # circle
viscosities = np.where(R < 0.2**2, 3.0, 1.0) # circle

_, ises, _ = dmAux.createFieldDecomposition()
# Might need to use the context manager here.
# With A as v:...v[0:nf] = ...
A[ises[0]] = densities
A[ises[1]] = viscosities

mesh.plex.compose("A", A)

#### Outputs

In [13]:
gVec = dmAux.createGlobalVector()
gVec.setName("auxiliary")
dmAux.localToGlobal(A, gVec)
viewer = PETSc.Viewer().createHDF5("auxiliary.h5", "w")
viewer(gVec)
viewer(dmAux)
del viewer
from underworld3.petsc_gen_xdmf import generateXdmf
generateXdmf("auxiliary.h5")

In [14]:
dmAux.destroy()
fe_viscosity.destroy()
fe_density.destroy()
fe_vel.destroy()
fe_pres.destroy()

In [15]:
#Create Closure index...(missing) (Added to petsc4py)
mesh.plex.createClosureIndex(None)

AttributeError: 'petsc4py.PETSc.DMPlex' object has no attribute 'createClosureIndex'

In [16]:
U = mesh.plex.createGlobalVector()
U.setName("solution")

### Set Initial guess

In [17]:
_, ises, _ = mesh.plex.createFieldDecomposition()

In [18]:
U[ises[0]] = np.zeros_like(ises[0]) # Zero Vector velocity
U[ises[1]] = np.ones_like(ises[1]) # One scalar for pressure

## Create Pressure Null Space

In [19]:
from underworld3.stokes import NullSpaceSetup

In [20]:
NullSpaceSetup(mesh.plex)

## Set SNES Convergence

In [21]:
snes.setConvergenceTest("default")

In [22]:
mesh.plex.setSNESLocalFEM()

In [23]:
snes.solve(None, U)

Error: error code 60
[0] SNESSolve() line 4459 in /tmp/petsc-build/petsc/src/snes/interface/snes.c
[0] SNESSolve_NEWTONLS() line 175 in /tmp/petsc-build/petsc/src/snes/impls/ls/ls.c
[0] SNESComputeFunction() line 2343 in /tmp/petsc-build/petsc/src/snes/interface/snes.c
[0] SNESComputeFunction_DMLocal() line 70 in /tmp/petsc-build/petsc/src/snes/utils/dmlocalsnes.c
[0] DMPlexSNESComputeBoundaryFEM() line 1673 in /tmp/petsc-build/petsc/src/snes/utils/dmplexsnes.c
[0] DMPlexInsertBoundaryValues() line 1019 in /tmp/petsc-build/petsc/src/dm/impls/plex/plexfem.c
[0] DMPlexInsertBoundaryValues_Plex() line 969 in /tmp/petsc-build/petsc/src/dm/impls/plex/plexfem.c
[0] DMPlexInsertBoundaryValuesEssential() line 754 in /tmp/petsc-build/petsc/src/dm/impls/plex/plexfem.c
[0] DMProjectFunctionLabelLocal() line 7724 in /tmp/petsc-build/petsc/src/dm/interface/dm.c
[0] DMProjectFunctionLabelLocal_Plex() line 707 in /tmp/petsc-build/petsc/src/dm/impls/plex/plexproject.c
[0] DMProjectLocal_Generic_Plex()

In [ ]:
U.destroy()
snes.destroy()
mesh.plex.destroy()